In [1]:
%load_ext Cython

Implement circumcircle radius, circumcenter, and helper functions in cython

In [10]:
%%cython
from libc.math cimport sqrt, fabs
import cython
import numpy as np
from scipy.spatial import cKDTree, ConvexHull

from metpy.gridding.polygons import order_edges, area

from metpy.gridding.triangles import find_local_boundary


def _area(pt1, pt2, pt3):
    
    cdef:
        float x0
        float y0
        float x1
        float y1
        float x2
        float y2
    
    x0, y0 = pt1
    x1, y1 = pt2
    x2, y2 = pt3
    
    return c_area(x0, y0, x1, y1, x2, y2)


cdef double c_area(float x0, float y0, float x1, float y1, float x2, float y2):
    
    cdef float a

    a = (x0 * y1 - x1 * y0) + (x1 * y2 - x2 * y1) + (x2 * y0 - x0 * y2)

    return fabs(a) * 0.5

def _circumcenter(pt0, pt1, pt2):


    cdef:
        float x0
        float y0
        float x1
        float y1
        float x2
        float y2
        float cx
        float cy
        
    x0, y0 = pt0
    x1, y1 = pt1
    x2, y2 = pt2

    c_circumcenter(&cx, &cy, x0, y0, x1, y1, x2, y2)
    
    return cx, cy


cdef void c_circumcenter(float * cx, float * cy, float x0, float y0,
                         float x1, float y1, float x2, float y2):
    
    cdef:
        float bc_y_diff
        float ca_y_diff
        float ab_y_diff
        float d_inv
        float a_mag
        float b_mag
        float c_mag

    bc_y_diff = y1 - y2
    ca_y_diff = y2 - y0
    ab_y_diff = y0 - y1
    
    d_inv = 0.5 / (x0 * bc_y_diff + x1 * ca_y_diff + x2 * ab_y_diff)
    
    a_mag = x0 * x0 + y0 * y0
    b_mag = x1 * x1 + y1 * y1
    c_mag = x2 * x2 + y2 * y2
    
    cx[0] = (a_mag * bc_y_diff + b_mag * ca_y_diff + c_mag * ab_y_diff) * d_inv
    cy[0] = (a_mag * (x2 - x1) + b_mag * (x0 - x2) + c_mag * (x1 - x0)) * d_inv
        

@cython.cdivision(True)
def _circumcircle_radius(pt1, pt2, pt3):
    
    cdef:
        float x0
        float y0
        float x1
        float y1
        float x2
        float y2
        float a
        
    x0, y0 = pt1
    x1, y1 = pt2
    x2, y2 = pt3
    
    a = _area(pt1, pt2, pt3)
    
    return c_circumcircle_radius(x0, y0, x1, y1, x2, y2, a)


cdef double c_dist_2(float x0, float y0, float x1, float y1):

    cdef:
        float d0
        float d1

    d0 = x1 - x0
    d1 = y1 - y0
    
    return d0 * d0 + d1 * d1

cdef double c_circumcircle_radius(float x0, float y0, float x1, 
                                  float y1, float x2, float y2,
                                  float area):

    cdef:
        float a
        float b
        float c
        float s
        float prod
        float radius
    
    radius = -99.00
    
    if area > 0:
        
        a = c_dist_2(x0, y0, x1, y1)
        b = c_dist_2(x1, y1, x2, y2)
        c = c_dist_2(x2, y2, x0, y0)

        prod = a * b * c
        
        radius = sqrt(prod) / (4 * area)

    return radius

def _find_natural_neighbors(tri, grid_points):

    tree = cKDTree(grid_points)

    in_triangulation = tri.find_simplex(tree.data) >= 0

    triangle_info = dict()

    members = dict((key, []) for key in range(len(tree.data)))

    for i in range(len(tri.simplices)):

        ps = tri.points[tri.simplices[i]]
        cc = _circumcenter(*ps)
        r = _circumcircle_radius(*ps)

        triangle_info[i] = {'cc': cc, 'r': r}

        qualifiers = tree.query_ball_point(cc, r)

        for qualifier in qualifiers:
            if in_triangulation[qualifier]:
                members[qualifier].append(i)

    return members, triangle_info

def _nn_point(xp, yp, variable, grid_loc, tri, neighbors, triangle_info):
    
    edges = find_local_boundary(tri, neighbors)
    edge_vertices = [segment[0] for segment in order_edges(edges)]
    num_vertices = len(edge_vertices)

    p1 = edge_vertices[0]
    p2 = edge_vertices[1]

    polygon = list()
    c1 = _circumcenter(grid_loc, tri.points[p1], tri.points[p2])
    polygon.append(c1)

    area_list = []
    total_area = 0.0

    for i in range(num_vertices):

        p3 = edge_vertices[(i + 2) % num_vertices]

        try:

            c2 = _circumcenter(grid_loc, tri.points[p3], tri.points[p2])
            polygon.append(c2)

            for check_tri in neighbors:
                if p2 in tri.simplices[check_tri]:
                    polygon.append(triangle_info[check_tri]['cc'])

        except ZeroDivisionError:
            area_list.append(0)
            continue

        pts = [polygon[i] for i in ConvexHull(polygon).vertices]
        value = variable[(tri.points[p2][0] == xp) & (tri.points[p2][1] == yp)]

        cur_area = area(pts)

        total_area += cur_area

        area_list.append(cur_area * value[0])

        polygon = list()
        polygon.append(c2)

        p1 = p2
        p2 = p3

    if total_area > 0:
        return sum([x / total_area for x in area_list])
    else:
        return np.nan

Implement the same functions in pure python

In [18]:
import math

from metpy.gridding import polygons
from metpy.gridding import triangles

def nn_point(xp, yp, variable, grid_loc, tri, neighbors, triangle_info):

    edges = triangles.find_local_boundary(tri, neighbors)
    edge_vertices = [segment[0] for segment in polygons.order_edges(edges)]
    num_vertices = len(edge_vertices)

    p1 = edge_vertices[0]
    p2 = edge_vertices[1]

    polygon = list()
    c1 = triangles.circumcenter(grid_loc, tri.points[p1], tri.points[p2])
    polygon.append(c1)

    area_list = []
    total_area = 0.0

    for i in range(num_vertices):

        p3 = edge_vertices[(i + 2) % num_vertices]

        try:

            c2 = triangles.circumcenter(grid_loc, tri.points[p3], tri.points[p2])
            polygon.append(c2)

            for check_tri in neighbors:
                if p2 in tri.simplices[check_tri]:
                    polygon.append(triangle_info[check_tri]['cc'])

        except ZeroDivisionError:
            area_list.append(0)
            continue

        pts = [polygon[i] for i in ConvexHull(polygon).vertices]
        value = variable[(tri.points[p2][0] == xp) & (tri.points[p2][1] == yp)]

        cur_area = polygons.area(pts)

        total_area += cur_area

        area_list.append(cur_area * value[0])

        polygon = list()
        polygon.append(c2)

        p1 = p2
        p2 = p3

    if total_area > 0:
        return sum([x / total_area for x in area_list])
    else:
        return np.nan

def circumcenter(pt0, pt1, pt2):

    a_x = pt0[0]
    a_y = pt0[1]
    b_x = pt1[0]
    b_y = pt1[1]
    c_x = pt2[0]
    c_y = pt2[1]

    bc_y_diff = b_y - c_y
    ca_y_diff = c_y - a_y
    ab_y_diff = a_y - b_y
    cb_x_diff = c_x - b_x
    ac_x_diff = a_x - c_x
    ba_x_diff = b_x - a_x

    d_div = (a_x * bc_y_diff + b_x * ca_y_diff + c_x * ab_y_diff)

    if d_div == 0:
        raise ZeroDivisionError

    d_inv = 0.5 / (a_x * bc_y_diff + b_x * ca_y_diff + c_x * ab_y_diff)

    a_mag = a_x * a_x + a_y * a_y
    b_mag = b_x * b_x + b_y * b_y
    c_mag = c_x * c_x + c_y * c_y

    cx = (a_mag * bc_y_diff + b_mag * ca_y_diff + c_mag * ab_y_diff) * d_inv
    cy = (a_mag * cb_x_diff + b_mag * ac_x_diff + c_mag * ba_x_diff) * d_inv

    return cx, cy


def triangle_area(pt1, pt2, pt3):

    a = 0.0
    
    x0, y0 = pt1
    x1, y1 = pt2
    x2, y2 = pt3

    a = (x0 * y1 - x1 * y0) + (x1 * y2 - x2 * y1) + (x2 * y0 - x0 * y2)

    return abs(a) / 2


def dist_2(x0, y0, x1, y1):
    d0 = x1 - x0
    d1 = y1 - y0
    return d0 * d0 + d1 * d1


def distance(p0, p1):
    return math.sqrt(dist_2(p0[0], p0[1], p1[0], p1[1]))


def circumcircle_radius(pt0, pt1, pt2):
    
    a = distance(pt0, pt1)
    b = distance(pt1, pt2)
    c = distance(pt2, pt0)

    t_area = triangle_area(pt0, pt1, pt2)

    if t_area > 0:
        prod = a * b * c

        radius = prod / (4 * t_area)
    else:
        radius = np.nan

    return radius

Same functions but compiled with numba.jit

In [19]:
import numba

numba.types.Array(float, 2, 'C')

array(<class 'float'>, 2d, C)

In [20]:
import math
import numba


def j_circumcenter(pt0, pt1, pt2):

    x0, y0 = pt0
    x1, y1 = pt1
    x2, y2 = pt2

    return jj_circumcenter(x0, y0, x1, y1, x2, y2)

@numba.jit("float64[2](float64, float64, float64, float64, float64, float64)")
def jj_circumcenter(x0, y0, x1, y1, x2, y2):
    
    bc_y_diff = y1 - y2
    ca_y_diff = y2 - y0
    ab_y_diff = y0 - y1
    
    d_inv = 0.5 / (x0 * bc_y_diff + x1 * ca_y_diff + x2 * ab_y_diff)
    
    a_mag = x0 * x0 + y0 * y0
    b_mag = x1 * x1 + y1 * y1
    c_mag = x2 * x2 + y2 * y2
    
    cx = (a_mag * bc_y_diff + b_mag * ca_y_diff + c_mag * ab_y_diff) * d_inv
    cy = (a_mag * (x2 - x1) + b_mag * (x0 - x2) + c_mag * (x1 - x0)) * d_inv
    
    return cx, cy

@numba.jit()
def j_triangle_area(pt1, pt2, pt3):

    a = 0.0
    
    x0, y0 = pt1
    x1, y1 = pt2
    x2, y2 = pt3

    a = (x0 * y1 - x1 * y0) + (x1 * y2 - x2 * y1) + (x2 * y0 - x0 * y2)

    return abs(a) / 2

@numba.jit("float64(float64, float64, float64, float64)")
def j_dist_2(x0, y0, x1, y1):
    d0 = x1 - x0
    d1 = y1 - y0
    return d0 * d0 + d1 * d1


def j_distance(p0, p1):
    return math.sqrt(j_dist_2(p0[0], p0[1], p1[0], p1[1]))

@numba.jit()
def j_circumcircle_radius(pt0, pt1, pt2):
    
    a = j_dist_2(pt0[0], pt0[1], pt1[0], pt1[1])
    b = j_dist_2(pt1[0], pt1[1], pt2[0], pt2[1])
    c = j_dist_2(pt2[0], pt2[1], pt0[0], pt0[1])

    t_area = j_triangle_area(pt0, pt1, pt2)

    if t_area > 0:
        prod = a * b * c

        radius = math.sqrt(prod) / (4 * t_area)
    else:
        radius = np.nan

    return radius

Use timeit to gauge the speed of the process

In [21]:
from numpy.testing import assert_almost_equal

p1 = [0.0, 0.0]
p2 = [10.1, 10.0]
p3 = [10.1, 0.0]

print("python circumcircle radius: ")
%timeit -n 500 circumcircle_radius(p1, p2, p3)

print("\ncython circumcircle radius: ")
%timeit -n 500 _circumcircle_radius(p1, p2, p3)

print("\nnumba circumcircle radius: ")
%timeit -n 500 j_circumcircle_radius(p1, p2, p3)

print("\npython circumcenter: ")
%timeit -n 500 circumcenter(p1, p2, p3)

print("\ncython circumcenter: ")
%timeit -n 500 _circumcenter(p1, p2, p3)

print("\nnumba circumcenter: ")
%timeit -n 500 j_circumcenter(p1, p2, p3)

assert_almost_equal(_circumcircle_radius(p1, p2, p3), circumcircle_radius(p1, p2, p3), 4)
assert_almost_equal(_circumcircle_radius(p1, p2, p3), j_circumcircle_radius(p1, p2, p3), 4)

assert_almost_equal(_circumcenter(p1, p2, p3), circumcenter(p1, p2, p3), 4)
assert_almost_equal(_circumcenter(p1, p2, p3), j_circumcenter(p1, p2, p3), 4)

print("\nCircumcircle Radius:", circumcircle_radius(p1, p2, p3))
print("\nCircumcenter:", circumcenter(p1, p2, p3))

python circumcircle radius: 
500 loops, best of 3: 4.91 µs per loop

cython circumcircle radius: 
500 loops, best of 3: 412 ns per loop

numba circumcircle radius: 
The slowest run took 273.55 times longer than the fastest. This could mean that an intermediate result is being cached.
500 loops, best of 3: 4.25 µs per loop

python circumcenter: 
500 loops, best of 3: 2.3 µs per loop

cython circumcenter: 
500 loops, best of 3: 271 ns per loop

numba circumcenter: 
500 loops, best of 3: 5.5 µs per loop

Circumcircle Radius: 7.106511098985211

Circumcenter: (5.05, 5.0)


In [22]:
def find_natural_neighbors(tri, grid_points):

    tree = cKDTree(grid_points)

    in_triangulation = tri.find_simplex(tree.data) >= 0

    triangle_info = dict()

    members = dict((key, []) for key in range(len(tree.data)))

    for i in range(len(tri.simplices)):

        ps = tri.points[tri.simplices[i]]
        
        cc = circumcenter(ps[0], ps[1], ps[2])
        r = circumcircle_radius(ps[0], ps[1], ps[2])

        triangle_info[i] = {'cc': cc, 'r': r}

        qualifiers = tree.query_ball_point(cc, r)

        for qualifier in qualifiers:
            if in_triangulation[qualifier]:
                members[qualifier].append(i)

    return members, triangle_info

In [28]:
from scipy.spatial import Delaunay

x = np.array(list(range(0, 20, 4)), dtype=float)
y = np.array(list(range(0, 20, 4)), dtype=float)
gx, gy = np.meshgrid(x, y)
pts = np.vstack([gx.ravel(), gy.ravel()]).T
tri = Delaunay(pts)

test_points = np.array([[2.0, 2.0], [5.0, 10.0], [12.0, 13.4], [12.0, 8.0], [20.0, 20.0]], dtype=float)

print("test natural find neighbors")
%timeit _find_natural_neighbors(tri, test_points)

%timeit find_natural_neighbors(tri, test_points)

test natural find neighbors
1000 loops, best of 3: 1.47 ms per loop
1000 loops, best of 3: 1.72 ms per loop


In [29]:
import numpy as np
from scipy.spatial import Delaunay

xp = np.array([8, 67, 79, 10, 52, 53, 98, 34, 15, 58])
yp = np.array([24, 87, 48, 94, 98, 66, 14, 24, 60, 16])

z = np.array([0.064, 4.489, 6.241, 0.1, 2.704, 2.809, 9.604, 1.156,
              0.225, 3.364])

tri = Delaunay(list(zip(xp, yp)))

sim_gridx = [30]
sim_gridy = [30]

members, tri_info = find_natural_neighbors(tri, list(zip(sim_gridx, sim_gridy)))

print("normal python nn for a point")
%timeit nn_point(xp, yp, z, [sim_gridx[0], sim_gridy[0]], tri, members[0], tri_info)

print("cython nn for a point")
%timeit _nn_point(xp, yp, z, [sim_gridx[0], sim_gridy[0]], tri, members[0], tri_info)

normal python nn for a point
1000 loops, best of 3: 967 µs per loop
cython nn for a point
1000 loops, best of 3: 936 µs per loop
